In [1]:
import requests
from io import StringIO
import pandas as pd

In [2]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [3]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

In [4]:
sentences[:4]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile']

In [5]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [6]:
for url in urls:
    res = requests.get(url)
    data_ = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    sentences.extend(data_[1].tolist())
    sentences.extend(data_[2].tolist())


In [7]:
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [8]:
sentences = [word for word in list(set(sentences)) if type(word) is str]

In [9]:
sentences_df = pd.DataFrame(sentences, columns=['sentence'])
sentences_df.head()

,sentence
0,The report by the independent expert committee...
1,combine so as to form a more complex product
2,A black dog is playing with a brown dog on the...
3,abstain from eating or from certain foods
4,A black and white photo of a living room with ...


In [10]:
data = pd.concat([data, sentences_df], axis=1)

In [11]:
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment,sentence
0,1.0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL,The report by the independent expert committee...
1,2.0,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL,combine so as to form a more complex product
2,3.0,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT,A black dog is playing with a brown dog on the...
3,5.0,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL,abstain from eating or from certain foods
4,9.0,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL,A black and white photo of a living room with ...


In [12]:
data.shape

(14504, 6)

# vectors embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')
sentences_embeddings = model.encode(sentences)
sentences_embeddings

In [14]:
import numpy as np
np.save('sentence_embeddings.npy', sentences_embeddings)

In [15]:
sentences_embeddings = np.load('sentence_embeddings.npy')

In [18]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 74.6 MB/s eta 0:00:00


In [19]:
import faiss

d = sentences_embeddings.shape[1]
print(d)

index = faiss.IndexFlatL2(d)
print(index.is_trained)

768
True


In [20]:
index.add(sentences_embeddings)

In [21]:
index.ntotal

14504

In [27]:
k = 4
xq = model.encode(["Someone sprits with a football"])

In [28]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 5354 12008  3436  4572]]
CPU times: user 6.14 ms, sys: 13 µs, total: 6.15 ms
Wall time: 6.47 ms


In [30]:
sentences_df.head()

,sentence
0,The report by the independent expert committee...
1,combine so as to form a more complex product
2,A black dog is playing with a brown dog on the...
3,abstain from eating or from certain foods
4,A black and white photo of a living room with ...


In [33]:
# sentences_df.iloc[I[0]]

In [ ]:
# [[5354 3436 4572 7562]]

In [32]:
data['sentence'].iloc[[ 5354 ,12008 , 3436 , 4572]]

,sentence
5354,Two groups of people are playing football
12008,Football players are on the field.
3436,A football player kicks the ball.
4572,position in football played by a team member


In [34]:
# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [36]:
d

768

In [35]:
print(vecs.shape)

print(vecs[0, :100])

(4, 768)
[ 0.22949545  0.21166402 -0.10311815 -0.08761431 -0.76231056  0.01426317
 -0.14125891  0.05316139 -1.28544343 -0.39343455 -1.13840997  0.35143641
  0.06776442  0.58293557  1.22395051  0.04434829 -0.19015428 -1.23700249
  0.30799544 -0.0492274  -0.94415843 -0.46993172 -0.74398804 -0.47364631
  0.53972477  0.30820453  0.44206688  0.42482728 -1.05848217  1.0223887
  0.30585548  0.23979108  0.44111779  0.39643267 -1.08022976 -0.80003738
  0.55200326 -0.69327897  0.38068026  0.22127882 -0.4883638   0.34378582
 -0.92080742  0.0873454  -0.73238271 -0.80773807 -0.97757173  0.26438352
 -1.00340438 -0.12847088  0.37874734  1.1222502  -1.70268917 -0.53649086
 -0.78519666  0.61356992  0.7391417  -0.73115408 -0.45036718  0.31849629
 -0.46307883 -0.2890403   0.14222956 -0.11835934 -0.96145844 -0.360493
 -0.03248365  0.08153825 -0.81519836 -0.88004899 -0.151976   -0.7504012
 -0.65406334 -0.20203713  0.27604499 -1.57993841  0.70451474  0.50699741
  0.50247443  0.09393804  0.09059737  0.601125

## Speed

- Using the IndexFlatL2 index alone is computationally expensive, it doesn’t scale well.

- When using this index, we are performing an exhaustive search — meaning we compare
- our query vector xq to every other vector in our index, in our case that is 14.5K L2-distance
- calculations for every search.

- For million or billions of embeddings, the index quickly becomes too slow to be useful, so we need to do something different.

## Partitioning The Index
- Faiss allows multiple optimizations of our search using different methods. A popular approach is to partition the index into Voronoi cells.

- Using this method, query vector xq identify the cell it belongs to, and then uses IndexFlatL2 or (another metric) to
search between the query vector and all other vector belonging to that specific cell.

- We reduce the scope of our search, producing an approximate answer, rather than exact (as produced by exhaustive search).

In [38]:
nlist = 50 # how many cells

quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

In [39]:
index.is_trained

False

- Train the index, such that grouping/transformation are applied on it. Here, we'll add clustering on that index.

In [40]:
index.train(sentences_embeddings)
print(index.is_trained)

index.add(sentences_embeddings)
print(index.ntotal)

True
14504


In [41]:
%%time

D , I = index.search(xq, k)
print(I)

[[5354 3436 4572 7562]]
CPU times: user 457 µs, sys: 876 µs, total: 1.33 ms
Wall time: 3.29 ms


- compare the time !!!

In [42]:
data['sentence'].iloc[[ 5354 ,3436  , 4572  , 7562]]

,sentence
5354,Two groups of people are playing football
3436,A football player kicks the ball.
4572,position in football played by a team member
7562,Two teams are competing in a football match


# Scoped Search after partitioning the index

- Previously, the probe was done on single cell. If approximate search with IndexFlatL2 returns suboptimal results, we can improve accuracy by increasing
the search scope. Let us increase the search scope by probing through 10 cells.

In [44]:
index.nprobe = 10

In [45]:
%%time

D,  I = index.search(xq, k)
print(I)

[[ 5354 12008  3436  4572]]
CPU times: user 2.11 ms, sys: 1.01 ms, total: 3.13 ms
Wall time: 3.21 ms


In [46]:
data['sentence'].iloc[[5354, 12008 , 3436 , 4572]]

,sentence
5354,Two groups of people are playing football
12008,Football players are on the field.
3436,A football player kicks the ball.
4572,position in football played by a team member


- Since we have increased the search scope to 10, we see a increase in time taken to search.
But compared to the first approach, this is still faster.

- If we go ahead and attempt to use index.reconstruct(<vector_idx>) again, we will return a RuntimeError
as there is no direct mapping between the original vectors and their index position, due to the addition of the IVF step.

In [47]:
index.make_direct_map()
index.reconstruct(7460)[:100]

array([-0.10073578, -0.5477037 , -0.81141806,  0.6375711 ,  0.06794177,
        0.3600828 , -0.2555709 ,  0.04699524,  0.03288049, -0.3587144 ,
       -0.1307556 ,  0.46006662,  0.43288168,  0.74488544,  0.84003294,
       -0.5014318 ,  0.9187963 , -0.39225712,  0.67373866, -0.17976218,
       -1.7720425 , -0.46576688, -1.0001596 , -0.42928773,  0.16651389,
        0.21137168,  0.38665426,  0.91876245,  0.2572644 , -0.5192197 ,
       -0.01203256,  1.4297947 ,  1.0217817 ,  0.5105299 , -0.07529938,
       -0.43335283, -0.5845765 , -1.1929612 ,  0.4843541 , -0.12222295,
        0.25022894, -1.0511798 , -0.24915949, -1.1119711 , -0.3302483 ,
       -0.53656447,  0.21086986,  0.14525287,  0.4352928 ,  0.1264186 ,
       -0.5146525 , -0.1266383 ,  0.16044486, -0.45585683, -0.5003363 ,
       -0.64471394,  0.2587578 , -0.04540078,  0.06427344,  0.0941467 ,
       -0.06161002, -0.4530893 ,  0.6897374 ,  0.44842562, -0.8656617 ,
        0.17439677,  1.0646037 , -0.53413445, -0.4432416 , -0.34

## Quantization
Each index stores the vectors as full (eg. Flat) vectors, we can compress the vectors using
product quantization.

PQ is an additional approximation step with a similar outcome to our use of IVF.
Where IVF allowed us to approximate by reducing the scope of our search,
PQ approximates the distance/similarity calculation instead.

PQ achieves this approximated similarity operation by compressing the vectors themselves, which consists of three steps.

image.png

- We split the original vector into several subvectors.
- For each set of subvectors, we perform a clustering operation — creating - multiple centroids for each sub-vector set.
- In our vector of sub-vectors, we replace each sub-vector with the ID of it’s nearest set-specific centroid.

In [48]:

m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [49]:
index.is_trained

False

In [50]:
index.train(sentences_embeddings)

In [51]:
index.is_trained

True

In [53]:
index.add(sentences_embeddings)

In [54]:
index.nprobe = 5  # align to previous IndexIVFFlat nprobe value

In [55]:
%%time
D, I = index.search(xq, k)
print(I)

[[1345 2940 8530  663]]
CPU times: user 872 µs, sys: 68 µs, total: 940 µs
Wall time: 708 µs


- time got further reduced !!!

In [56]:
[f'{i}: {sentences[i]}' for i in I[0]]

['1345: A football player in a red and white uniform is being lifted by two teammates',
 '2940: Two teammates are lifting a football player in a red and white uniform',
 '8530: A football player is carrying an official past a rolling football',
 '663: Three football players from red team tackling one player from white team.']

- By adding PQ, we have reduced our time from milliseconds to microseconds. Minor
changes in results are expected, thus both of our speed optimization operations,
IVF and PQ, come at the cost of accuracy.